In [ ]:
# Selenium 라이브러리 설치
!pip install selenium
from selenium import webdriver

In [ ]:
# 패키지와 드라이버 설치
import sys
!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin/

In [ ]:
# Selenium 관련 환경 설정
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, InvalidSessionIdException
from time import sleep
from bs4 import BeautifulSoup
import re
import json

sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('disable-dev-shm-usage')
options.add_experimental_option("detach", True)

webdriver_service = Service('/usr/bin/chromedriver')
driver = webdriver.Chrome(service=webdriver_service, options=options)

In [ ]:
# 웹페이지 파싱
url = "https://map.naver.com/v5/search"
driver.get(url)
key_word = '신촌 맛집'
print(driver)

<selenium.webdriver.chrome.webdriver.WebDriver (session="42f40fe4a9b6723ed358898671ee44a3")>


In [ ]:
# css 찾을때 까지 10초 대기
def time_wait(num, code):
  try:
    wait = WebDriverWait(driver, num).until(
      EC.presence_of_element_located((By.CSS_SELECTOR, code)))
  except:
    print(code, '태그를 찾지 못하였습니다.')
    driver.quit()
  return wait

In [ ]:
# frame 변경 메소드
def switch_frame(frame):
  driver.switch_to.default_content()  # frame 초기화
  sleep(1)
  driver.switch_to.frame(frame)  # frame 변경
  res
  soup

In [ ]:
# 페이지 다운
def page_down(num):
  body = driver.find_element(By.CSS_SELECTOR, 'body')
  body.click()
  for i in range(num):
    body.send_keys(Keys.PAGE_DOWN)

In [ ]:
time_wait(10, 'div.input_box > input.input_search')

# (1) 검색창 찾기
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

sleep(10)

res = driver.page_source  # 페이지 소스 가져오기
soup = BeautifulSoup(res, 'html.parser')  # html 파싱

# (2) frame 변경 & 페이지 스크롤 다운
switch_frame("searchIframe")
page_down(60)
sleep(5)

In [ ]:
store_list = WebDriverWait(driver, 10).until(
                  EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.UEzoS')))
next_btn = driver.find_elements(By.CSS_SELECTOR, '.mBN2s')
store_dict = []

print(len(store_list), len(next_btn))

70 5


In [ ]:
# 시작시간
start = time.time()
print('[크롤링 시작...]')

try:
    for btn in range(len(next_btn))[1:]:
        for data in range(len(store_list)):  # 매장 리스트 만큼 반복
            # (3) 상세 페이지로 이동 (페이지 클릭)
            try:
                page = driver.find_elements(By.CSS_SELECTOR, '.TYaxT')
                if data < len(page):
                    page[data].click()
                    time.sleep(2)

                switch_frame('entryIframe')
                time_wait(5, '.GHAhO')

                # 페이지 스크롤 다운
                for _ in range(2):
                    time.sleep(1)
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # ----- (4) 매장명 & 장소 유형 -----
                store_name = driver.find_element(By.CSS_SELECTOR, '.GHAhO').text
                store_type = driver.find_element(By.CSS_SELECTOR, '.lnJFt').text

                # ----- (5) 별점 -----
                try:
                    store_rating = driver.find_element(By.CSS_SELECTOR, '.PXMot.LXIwF').text
                except:
                    store_rating = -1

                # ----- (6) 리뷰 탭으로 이동 -----
                try:
                    # veBoZ 클래스를 가진 모든 요소 찾기
                    tabs = driver.find_elements(By.CSS_SELECTOR, '.veBoZ')

                    # 각 탭 요소를 확인하여 텍스트가 "리뷰"인 탭 클릭
                    for tab in tabs:
                        if tab.text == "리뷰":
                            tab.click()
                            time.sleep(2)
                            break

                    # ----- (7) 대표 키워드 1 -----
                    keyword1 = driver.find_elements(By.CSS_SELECTOR, '.t3JSf')[0].text

                    # ----- (7) 대표 키워드 2 -----
                    keyword2 = driver.find_elements(By.CSS_SELECTOR, '.t3JSf')[1].text

                    # ----- 방문자 리뷰 -----
                    all_reviews = []
                    try:
                        # (8) 더보기 버튼을 클릭하여 리뷰 추가로 로드 - 더보기 한 번 당 10개의 리뷰
                        for _ in range(1):
                            try:
                                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                                sleep(3)
                                driver.find_element(By.CSS_SELECTOR, '.fvwqf').click()
                                sleep(3)
                                driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)

                            except NoSuchElementException:
                                print('-더보기 버튼 모두 클릭 완료-')
                                break

                        reviews = WebDriverWait(driver, 10).until(
                                      EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.zPfVt')))

                        # 각 리뷰 텍스트를 가져와서 리스트에 추가
                        for review in reviews:
                            try:
                                # (9)-a. 토글 버튼 클릭하여 전체 리뷰 텍스트 표시
                                toggle_button = review.find_element(By.CSS_SELECTOR, '.Ky28p')
                                toggle_button.click()
                                time.sleep(2)  # 토글 버튼 클릭 후 잠시 대기
                                full_text = review.find_element(By.CSS_SELECTOR, '.zPfVt').text
                                all_reviews.append(full_text)
                            except:
                                # (9)-b. 토글 버튼이 없는 경우 그냥 리뷰 텍스트 가져오기
                                text = review.text
                                all_reviews.append(text)

                    except:
                        all_reviews.append(' ')

                except:
                    print("리뷰탭을 찾지 못함")

                # 매장 정보 딕셔너리에 추가
                dict_temp = {
                              'name': store_name,
                              'type': store_type,
                              'rating' : store_rating,
                              '1st_keyword' : keyword1,
                              '2ed_keyword' : keyword2,
                              'review' : all_reviews
                            }

                store_dict.append(dict_temp)
                print(f'{store_name} ...완료')

                # 검색 iframe으로 전환
                switch_frame('searchIframe')
                time.sleep(1)

            except Exception as e:
                print(f"Error occurred while processing store data: {e}")

        # (10) 다음 페이지 버튼 클릭
        try:
            next_button = driver.find_elements(By.CSS_SELECTOR, '.mBN2s')
            if next_button[btn]:
                next_button[btn].click()
                time.sleep(5)
        except InvalidSessionIdException as e:
            print(f"Invalid session ID exception: {e}")
            break
        except Exception as e:
            print(f"Error occurred while clicking next button: {e}")
            break

finally:
    print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
    driver.quit()

# json 파일로 저장
with open('/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/store_data.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

In [ ]:
import json

# JSON 파일 경로 설정
file_path = '/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/store_data.json'

# JSON 파일 열기
with open(file_path, 'r') as file:
    data = json.load(file)
data

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv("/content/drive/MyDrive/[2024-1] EURON 6기/프로젝트/data/store_data.csv", index=False)

In [ ]:
df.tail(20)

,name,type,rating,1st_keyword,2ed_keyword,review
79,카린지린가네스낵바 신촌점,요리주점,-1,"""음식이 맛있어요""","""인테리어가 멋져요""",[ ]
80,롤링파스타 신촌점,"스파게티,파스타전문",별점\n4.33,"""음식이 맛있어요""","""가성비가 좋아요""",[○앉은자리가 주문태블릿도 없고 구석진 자리라 주문하기 어려웠음 ○음식 나오는 시간...
81,더도이축산직영점 신촌점,"육류,고기요리",별점\n4.34,"""음식이 맛있어요""","""가성비가 좋아요""","[웨이팅이 길어서 얼마나 맛있나싶었는데, 기다릴만합니다..삼겹살이 존맛이네요..😹,..."
82,신촌서서갈비,"육류,고기요리",별점\n4.19,"""음식이 맛있어요""","""친절해요""",[싱가폴에서 친구가 1년만에 나와서 몇달만에 들른 신촌서서갈비 역시나 고기도 야들야...
83,신촌장군집,"육류,고기요리",-1,"""음식이 맛있어요""","""가성비가 좋아요""",[ ]
84,삼미식당 신촌점,"육류,고기요리",별점\n4.46,"""음식이 맛있어요""","""친절해요""",[친구들과 운동후 고기흡수위해 예약하고 방문.\n자주가는곳이라 고기질은 말할필요없죠...
85,그레인바운더리 신촌,베이커리,-1,"""인테리어가 멋져요""","""커피가 맛있어요""","[사진 잘나와요. 디저트 커피 다 맛있어요., 신촌에 오면 꼭 오세요 ♡ 쿠키대박,..."
86,히츠지야 신촌점,요리주점,-1,"""음식이 맛있어요""","""인테리어가 멋져요""",[일단 인테리어가 일본 분위기 나서 일본 여행온 느낌나서 너무 좋았구요! 훗카이도 ...
87,타코벨 신촌점,"멕시코,남미음식",별점\n4.4,"""음식이 맛있어요""","""혼밥하기 좋아요""",[타코벨 오랜만에 먹었는데 맛있고 구성이 좋았어요! 2인이 같이 먹기에 좋구 포장인...
88,방콕익스프레스 신촌본점,태국음식,별점\n4.36,"""음식이 맛있어요""","""양이 많아요""","[늘 맛있게 먹는 애정하는 식당\n친구도 맛있다며 또 오겠다네요ㅎㅎ, 새우 팟타이 ..."
